In [2]:
# Это пример выгрузки, а также обработки некоторых строковых данных и изменение типа

import pandas as pd
import matplotlib.pyplot as plt
import re
import ast
import numpy as np
import seaborn as sns

df = pd.read_csv('../mvideo_features.csv')

df = df.drop_duplicates(subset=df.columns.difference(['product_id']), keep='first').reset_index(drop=True)
print(f'Размерность после удаления дубликатов: {df.shape}')

Размерность после удаления дубликатов: (929, 104)


In [5]:
df1 = df.copy()
df1.head(3)

,product_id,name,brand,Гарантия,Гарантия предоставляется,Страна,Год релиза,Серия,Состояние,Внешний вид,...,Разрешение дополнительного экрана по вертикали (Пикс),Разрешение дополнительного экрана по горизонтали (Пикс),Складной смартфон,Встроенный искусственный интеллект,Проводная гарнитура,Micro-SIM,Гарантия на аксессуары (мес.),Гарантия на АКБ (мес.),Дополнительная комплектация,Сканер радужной оболочки глаза
0,30063534,Смартфон Apple iPhone 13 128GB nanoSim/eSim Mi...,Apple,1 год,продавцом,Китай,2021.0,iPhone 13,новый,новый,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,400410049,Смартфон HONOR X9c 12/256GB Titanium Black,HONOR,2 года,производителем,Китай,2025.0,X9c,новый,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,400307033,Смартфон HUAWEI Pura 70 Pro 12/512GB White,HUAWEI,1 год,производителем,Китай,2024.0,Pura 70 Pro,новый,новый,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def choose(row):
    if '0' in str(row['memory2']):
        return row['memory2']
    if '0' in str(row['memory3']):
        return row['memory3']
    return row['memory4']

def get_memory(x):
    if type(x) == float:
        return x
    if type(x) == str:
        f = re.findall(r'[0-9]+', x)
        if type(f[-1]) == str:
            return float(f[-1])
    return x
def get_memory_from_name(x):
    if 'TB' in x:
        f = re.findall(r'[0-9/]+TB', x)
        return get_memory(f[0]) * 1024.0 if len(f) > 0 else np.nan
    if 'GB' in x:
        f = re.findall(r'[0-9/]+GB', x)
        return get_memory(f[0]) if len(f) > 0 else np.nan
    return np.nan

df1['memory2'] = df1["Оперативная и встроенная память"].apply(lambda x: get_memory(x))
df1['memory3'] = df1["Встроенная память (ROM) (ГБ)"].apply(lambda x: get_memory(x))
df1['memory4'] = df1["name"].apply(lambda x: get_memory_from_name(x))
df1['memory'] = df1.apply(choose, axis=1)

df1 = df1.drop(columns=["Оперативная и встроенная память", "Встроенная память (ROM) (ГБ)", 'memory2', 'memory3', 'memory4'])


0       128.0
1       256.0
2       512.0
3       256.0
4       512.0
        ...  
924    1024.0
925     256.0
926     512.0
927     512.0
928     256.0
Name: memory, Length: 929, dtype: float64

In [8]:
def get_height(x):
    if type(x) == str:
        f = re.findall(r'[0-9.]+', x)
        if type(f[0]) == str:
            return float(f[0])
    return x

def get_width(x):
    if type(x) == str:
        f = re.findall(r'[0-9.]+', x)
        if type(f[1]) == str:
            return float(f[1])
    return x


def get_depth(x):
    if type(x) == str:
        f = re.findall(r'[0-9.]+', x)
        if len(f) < 3:
            return np.nan
        if type(f[2]) == str:
            return float(f[2])
    return np.nan

def choose_h(row):
    if row['height1'] !=  np.nan:
        return row['height1']
    return row['height2']
def choose_w(row):
    if row['width1'] !=  np.nan:
        return row['width1']
    return row['width2']
def choose_d(row):
    if row['depth1'] !=  np.nan:
        return row['depth1']
    return row['depth2']


df1['height1'] = df1["Габаритные размеры (В*Ш*Т)"].apply(lambda x: get_height(x))
df1['width1'] = df1["Габаритные размеры (В*Ш*Т)"].apply(lambda x: get_width(x))
df1['depth1'] = df1["Габаритные размеры (В*Ш*Т)"].apply(lambda x: get_depth(x))

df1['height2'] = df1["Габаритные размеры (В*Ш*Г)"].apply(lambda x: get_height(x))
df1['width2'] = df1["Габаритные размеры (В*Ш*Г)"].apply(lambda x: get_width(x))
df1['depth2'] = df1["Габаритные размеры (В*Ш*Г)"].apply(lambda x: get_depth(x))

df1['height'] = df1.apply(choose_h, axis=1)
df1['width'] = df1.apply(choose_w, axis=1)
df1['depth'] = df1.apply(choose_d, axis=1)

df1['height'].fillna((df1['height'].mean()), inplace=True)
df1['width'].fillna((df1['width'].mean()), inplace=True)
df1['depth'].fillna((df1['depth'].mean()), inplace=True)

df1['square'] = df1['height'] * df1['width']
df1['volume'] = df1['height'] * df1['width'] * df1['depth']

df1 = df1.drop(columns=["Габаритные размеры (В*Ш*Т)", "Габаритные размеры (В*Ш*Г)", 
                        'height1', 'height2', 'width1', 'width2', 'depth1', 'depth2'])


In [13]:
def get_res_0(x):
    if type(x) == str:
        f = re.findall(r'[0-9.]+', x)
        if type(f[0]) == str:
            return float(f[0])
    return 720.0  # in case none, probably the phone still has 720 as resolution

def get_res_1(x):
    if type(x) == str:
        f = re.findall(r'[0-9.]+', x)
        if type(f[1]) == str:
            return float(f[1])
    return 720.0



df1['video_resolution_v'] = df1["Разрешение видеосъемки"].apply(lambda x: get_res_1(x))
df1['video_resolution_h'] = df1["Разрешение видеосъемки"].apply(lambda x: get_res_0(x))

df1['screen_resolution_h'] = df1["Разрешение экрана"].apply(lambda x: get_res_0(x))
df1['screen_resolution_v'] = df1["Разрешение экрана"].apply(lambda x: get_res_1(x))

df1['screen_diag'] = df1["Экран"].apply(lambda x: np.nan if get_res_0(x) > 9.0 else get_res_0(x))
df1['screen_diag'].fillna((df1['screen_diag'].mean()), inplace=True)


df1 = df1.drop(columns=["Разрешение видеосъемки", "Разрешение экрана", "Экран"])


/var/folders/wt/lsf6j96n0xnc67mjwrs1l5p00000gn/T/ipykernel_74229/2849039046.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['screen_diag'].fillna((df1['screen_diag'].mean()), inplace=True)


In [18]:
def get_dust_IP(x):
    if type(x) is str and x[:2] == 'IP':
        if x[2] == 'X':
            return 0.0
        return int(x[2])
    return 0.0  # in case if nan, we will give the minimum dust_IP value

def get_water_IP(x):
    if type(x) is str and  x[:2] == 'IP':
        if x[3] == 'X':
            return 0.0
        return int(x[3])
    return 0.0 # in case if nan, we will give the minimum water_IP value


df1['dust_IP'] = df1["Степень защиты"].apply(lambda x: get_dust_IP(x))
df1['water_IP'] = df1["Степень защиты"].apply(lambda x: get_water_IP(x))

df1 = df1.drop(columns=["Степень защиты"])


In [20]:
df1['zoom'] = df1['Цифровой зум (x)']
df1['weight'] = df1['Вес (г)']
df1['kernel_cnt'] = df1['Количество ядер']
df1['screen_technology'] = df1['Технология экрана']
df1['power'] = df1['Мощность блока питания (Вт)']

df1['weight'].fillna((df1['weight'].mean()), inplace=True)

df1 = df1.drop(columns=["Цифровой зум (x)", "Вес (г)", "Количество ядер", "Технология экрана", "Мощность блока питания (Вт)"])

/var/folders/wt/lsf6j96n0xnc67mjwrs1l5p00000gn/T/ipykernel_74229/4147641621.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['weight'].fillna((df1['weight'].mean()), inplace=True)


In [32]:
bin_size = 500


df1['battery_bin'] = pd.to_numeric(df1['Емкость аккумулятора (мАч)'], errors='coerce')
df1['battery_bin'].fillna((bin_size), inplace=True)
df1['is_5G'] = df['Поддержка стандартов'].apply(lambda x: int('5G' in x) if type(x) == str else 0)
df1['battery_bin'] = (df1['battery_bin'] // bin_size * bin_size).astype(int)

df1 = df1.drop(columns=['Емкость аккумулятора (мАч)'])

/var/folders/wt/lsf6j96n0xnc67mjwrs1l5p00000gn/T/ipykernel_74229/956400669.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['battery_bin'].fillna((bin_size), inplace=True)


In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

X = df1.drop('salePrice', axis=1)
y = df1['salePrice']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Предварительная обработка данных
numeric_features = ['memory', 'height', 'width', 'depth', 'square', 'video_resolution_v', 'video_resolution_h',
                    'screen_resolution_h', 'screen_resolution_v', 'screen_diag', 'weight', 'battery_bin']
categorical_features = ['brand', 'Материал корпуса', 'is_5G', 'Год релиза', 'Поддержка быстрой зарядки',
                        'Поддержка беспроводной зарядки', 'Гарантия предоставляется', 'Страна']

# Масштабирование числовых признаков
scaler = StandardScaler()

X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])


# One-hot encoding категориальных признаков
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = pd.get_dummies(X_train[categorical_features], drop_first=False)
X_test_encoded = pd.get_dummies(X_train[categorical_features], drop_first=False)

X_train_encoded

# # Объединение числовых и категориальных признаков
# X_train_final = pd.concat([X_train[numeric_features], pd.DataFrame(X_train_encoded.toarray())], axis=1)
# X_test_final = pd.concat([X_test[numeric_features], pd.DataFrame(X_test_encoded.toarray())], axis=1)



# X_train_final = X_train_final.rename(str,axis="columns") 
# X_test_final = X_test_final.rename(str,axis="columns") 

# X_train_final

# # X_train_final

# # Обучение моделей
# models = {
#     'Linear Regression': LinearRegression(),
#     'SVR': SVR(kernel='rbf'),
#     'Random Forest': RandomForestRegressor(n_estimators=100),
#     'Gradient Boosting': GradientBoostingRegressor(n_estimators=100)
# }

# results = {}
# for name, model in models.items():
#     model.fit(X_train_final, y_train)
#     y_pred = model.predict(X_test_final)
    
#     mae = mean_absolute_error(y_test, y_pred)
#     mse = mean_squared_error(y_test, y_pred)
    
#     results[name] = {'MAE': mae, 'MSE': mse}
    
#     print(f"{name}:")
#     print(f"MAE: {mae:.2f}")
#     print(f"MSE: {mse:.2f}\n")


,is_5G,Год релиза,brand_Apple,brand_HONOR,brand_HUAWEI,brand_Infinix,brand_Nothing,brand_Nubia,brand_OPPO,brand_OnePlus,...,Материал корпуса_стекло/ пластик,Материал корпуса_титан/ пластик,Материал корпуса_титан/ стекло,Поддержка быстрой зарядки_Да,Поддержка беспроводной зарядки_Да,Гарантия предоставляется_продавцом,Гарантия предоставляется_производителем,Страна_Вьетнам,Страна_Китай,Страна_ОАЭ
381,1,NaN,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,True,False,True,False
499,1,2021.0,True,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,True,False
97,0,2024.0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
250,1,2023.0,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
198,0,2023.0,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0,2024.0,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
270,0,2025.0,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
860,1,2024.0,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,True,True,False,False
435,0,2024.0,False,False,False,True,False,False,False,False,...,False,False,False,True,True,False,True,False,True,False
